In [2]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'mesolitica-tpu.json'

In [3]:
from glob import glob
import tensorflow as tf
from tqdm import tqdm
import malaya_speech
from malaya_speech.utils import subword
import numpy as np
import mp
from google.cloud import storage

/home/ubuntu/.local/lib/python3.7/site-packages/malaya_boilerplate/frozen_graph.py:25: UserWarning: Cannot import beam_search_ops from Tensorflow Addons, `deep_model` for stemmer will not available to use, make sure Tensorflow Addons version >= 0.12.0
  'Cannot import beam_search_ops from Tensorflow Addons, `deep_model` for stemmer will not available to use, make sure Tensorflow Addons version >= 0.12.0'


In [5]:
singlish = glob('call-centre-1/wav/*.wav')
len(singlish)

388081

In [6]:
subwords = subword.load('transducer-singlish.subword')

In [7]:
import unicodedata
import re
import itertools

vocabs = [" ", "a", "e", "n", "i", "t", "o", "u", "s", "k", "r", "l", "h", "d", "m", "g", "y", "b", "p", "w", "c", "f", "j", "v", "z", "0", "1", "x", "2", "q", "5", "3", "4", "6", "9", "8", "7"]

def preprocessing_text(string):
    
    string = unicodedata.normalize('NFC', string.lower())
    string = string.replace('\'', '')
    string = ''.join([c if c in vocabs else ' ' for c in string])
    string = re.sub(r'[ ]+', ' ', string).strip()
    string = (
        ''.join(''.join(s)[:2] for _, s in itertools.groupby(string))
    )
    return string

In [8]:
def get_after_mandarin(word):
    if '<mandarin>' in word:
        w = word.split('>')[1].split(':')[1]
        return w.split('</')[0]
    else:
        return word
    
def get_before_mandarin(word):
    if '</mandarin>' in word:
        return word.split('</')[0]
    else:
        return word

def replace_paralinguistic(string, replaces = ['(ppb)', '(ppc)', '(ppl)', '(ppo)', '<UNK>', '<MANDARIN>']):
    for r in replaces:
        string = string.replace(r, ' ')
    string = string.split()
    string = [get_after_mandarin(w) for w in string]
    string = [get_before_mandarin(w) for w in string]
    string = [w for w in string if w[0] not in '<[(' and w[-1] not in '>])']
    return ' '.join(string)

In [9]:
def loop(files):
    files, index = files
    results = []
    for i in tqdm(files):
        try:
            p = i.replace('/wav','/text')
            with open(f'{p}.txt') as fopen:
                text = fopen.read()
            if len(text) < 2:
                continue
            if text[0] == '<' and text[-1] == '>':
                continue
            text = replace_paralinguistic(text)
            text = preprocessing_text(text)
            if len(text):
                results.append((i, text))
        except Exception as e:
            pass
    return results

In [10]:
loop((singlish[:10], 0))

100%|██████████| 10/10 [00:00<00:00, 558.01it/s]


[('call-centre-1/wav/app_0489_0009_phnd_cc-hot-0-0.wav',
  'one two three hotel'),
 ('call-centre-1/wav/app_0169_3338_phnd_cc-hot-0-37.wav', 'yes i am'),
 ('call-centre-1/wav/app_0388_3776_phnd_cc-hol-0-132.wav',
  'no qualms about that its only this'),
 ('call-centre-1/wav/app_0523_0018_phnd_cc-res-0-109.wav',
  'okay sure i will put put it in as well'),
 ('call-centre-1/wav/app_0155_0001_phnd_cc-hot-0-52.wav',
  'okay move on to negative feedback and in'),
 ('call-centre-1/wav/app_0547_0001_phnd_cc-hol-0-70.wav', 'ya'),
 ('call-centre-1/wav/app_0638_4276_phnd_cc-hol-0-65.wav',
  'and my in and then seems to be not interested in my enq enquiry')]

In [11]:
singlishs = mp.multiprocessing(singlish, loop, cores = 12)

100%|██████████| 32340/32340 [06:37<00:00, 81.41it/s]  


In [12]:
len(singlishs)

238343

In [13]:
import six

def to_example(dictionary):
    """Helper: build tf.Example from (string -> int/float/str list) dictionary."""
    features = {}
    for (k, v) in six.iteritems(dictionary):
        if not v:
            raise ValueError('Empty generated field: %s' % str((k, v)))
        # Subtly in PY2 vs PY3, map is not scriptable in py3. As a result,
        # map objects will fail with TypeError, unless converted to a list.
        if six.PY3 and isinstance(v, map):
            v = list(v)
        if isinstance(v[0], six.integer_types) or np.issubdtype(
            type(v[0]), np.integer
        ):
            features[k] = tf.train.Feature(
                int64_list=tf.train.Int64List(value=v)
            )
        elif isinstance(v[0], float):
            features[k] = tf.train.Feature(
                float_list=tf.train.FloatList(value=v)
            )
        elif isinstance(v[0], six.string_types):
            if not six.PY2:  # Convert in python 3.
                v = [bytes(x, 'utf-8') for x in v]
            features[k] = tf.train.Feature(
                bytes_list=tf.train.BytesList(value=v)
            )
        elif isinstance(v[0], bytes):
            features[k] = tf.train.Feature(
                bytes_list=tf.train.BytesList(value=v)
            )
        else:
            raise ValueError(
                'Value for %s is not a recognized type; v: %s type: %s'
                % (k, str(v[0]), str(type(v[0])))
            )
    return tf.train.Example(features=tf.train.Features(feature=features))

In [14]:
sr = 16000
maxlen = 18
minlen_text = 1
global_count = 0

In [15]:
def loop(files):
    client = storage.Client()
    bucket = client.bucket('mesolitica-tpu-general')
    files, index = files
    output_file = f'{index}-{global_count}.tfrecord'
    writer = tf.io.TFRecordWriter(output_file)
    for s in tqdm(files):
        try:
            if len(s[1]) < minlen_text:
                continue
            y, _ = malaya_speech.load(s[0])
            if (len(y) / sr) > maxlen:
                continue
            t = subword.encode(subwords, s[1], add_blank=False)
            example = to_example({'waveforms': y.tolist(), 
                                  'targets': t, 
                                  'targets_length': [len(t)]})
            writer.write(example.SerializeToString())
        except Exception as e:
            print(e)
            pass
    writer.close()
    blob = bucket.blob(f'imda/part6-call-centre-1/{output_file}')
    blob.upload_from_filename(output_file)
    os.system(f'rm {output_file}')

In [16]:
loop((singlishs[:10], 0))

100%|██████████| 10/10 [00:00<00:00, 17.31it/s]


In [17]:
batch_size = 25000
for i in range(0, len(singlishs), batch_size):
    batch = singlishs[i: i + batch_size]
    mp.multiprocessing(batch, loop, cores = 6, returned = False)
    global_count += 1

  2%|▏         | 82/4166 [00:05<03:59, 17.04it/s]

zero-size array to reduction operation maximum which has no identity


  6%|▌         | 233/4166 [00:16<03:45, 17.42it/s]

zero-size array to reduction operation maximum which has no identity

  5%|▌         | 217/4166 [00:16<05:54, 11.14it/s]

...

zero-size array to reduction operation maximum which has no identity

 80%|███████▉  | 1772/2223 [02:00<00:30, 14.60it/s]

 88%|████████▊ | 1967/2223 [02:12<00:16, 15.56it/s]

zero-size array to reduction operation maximum which has no identity


100%|██████████| 5/5 [00:00<00:00, 57.10it/s]


In [18]:
from malaya_speech.utils import tf_featurization

config = malaya_speech.config.transducer_featurizer_config
featurizer = tf_featurization.STTFeaturizer(**config)

In [19]:
n_mels = 80

def preprocess_inputs(example):
    s = featurizer.vectorize(example['waveforms'])
    mel_fbanks = tf.reshape(s, (-1, n_mels))
    example['inputs'] = mel_fbanks
    return example

def parse(serialized_example):

    data_fields = {
        'waveforms': tf.compat.v1.VarLenFeature(tf.float32),
        'targets': tf.compat.v1.VarLenFeature(tf.int64),
        'targets_length': tf.compat.v1.VarLenFeature(tf.int64),
    }
    features = tf.compat.v1.parse_single_example(
        serialized_example, features = data_fields
    )
    for k in features.keys():
        features[k] = features[k].values
        
    features = preprocess_inputs(features)

    keys = list(features.keys())
    for k in keys:
        if k not in ['waveforms', 'inputs', 'targets', 'targets_length']:
            features.pop(k, None)

    return features

def get_dataset(files, batch_size = 2, shuffle_size = 32, thread_count = 24):
    def get():
        dataset = tf.data.TFRecordDataset(files)
        dataset = dataset.shuffle(shuffle_size)
        dataset = dataset.map(parse, num_parallel_calls = thread_count)
        dataset = dataset.repeat()
        return dataset

    return get

In [20]:
files = tf.io.gfile.glob('gs://mesolitica-tpu-general/imda/part6-call-centre-1/*.tfrecord')
d = get_dataset(files)()
d = d.as_numpy_iterator()

In [21]:
next(d)

{'targets': array([  8, 795,  69,  16,   8, 795,   1,   9, 172, 432,   4,  34,  29,
        795, 106, 407, 278,  11,  16,  69, 168, 876, 795, 168, 279, 305]),
 'targets_length': array([26]),
 'waveforms': array([0.02579204, 0.01462226, 0.00355402, ..., 0.00863119, 0.01045898,
        0.00781885], dtype=float32),
 'inputs': array([[-3.259451  , -3.2115316 , -3.1417866 , ..., -0.4800198 ,
         -0.79051363, -0.6523645 ],
        [-3.0369134 , -2.9883745 , -2.9177563 , ..., -0.71144384,
         -0.59316975, -0.6146458 ],
        [-3.3031645 , -2.9903703 , -2.7309544 , ..., -0.7843823 ,
         -0.5416616 , -0.39892134],
        ...,
        [-4.557473  , -4.1015425 , -3.8034575 , ..., -0.93814534,
         -0.85005796, -0.51457846],
        [-3.3352256 , -2.9848642 , -2.713126  , ..., -0.8856256 ,
         -0.53133416, -0.7989203 ],
        [-3.8264012 , -3.278942  , -2.9650037 , ..., -0.65680426,
         -0.5076165 , -0.9370476 ]], dtype=float32)}